In [9]:
import pandas as pd

df = pd.read_csv("D:/code/2ème année/projet-python/data/prix-carburants-fichier-quotidien-test-ods.csv", sep=';', on_bad_lines='skip')

def data_carb(df, carb):
    data_1 = df[['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom']]
    d = data_1[data_1.prix_nom == carb]
    #d=d.drop_duplicates(keep = 'first', inplace=True) 
    #df.drop_duplicates(subset ="'column'", keep = 'first', inplace=True) si on veut supprimer selon une colonne

# a ajouter : un traitement des NaN
    return d

df_2 = data_carb(df, 'SP95').sort_values('prix_valeur')
df_test=df_2.sample(5)

In [24]:
from geopy.geocoders import Nominatim


def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="Projet Python")
    location = geolocator.geocode(Adresse)
    return str(str(location.latitude) + ','+ str(location.longitude))

from math import cos, sin, acos 

def distance1(p1,p2):
    lat1,long1 = p1
    lat2,long2 = p2
    dlong = long2 - long1
    dist_ang = acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(dlong))
    return(6378137*dist_ang)

print(Adresse_to_gps("1 rue d'Arsonval, Orsay, France"))

48.7087952,2.1742033


In [25]:
df_opti=df_test.assign(cout=1) #nouvelle colonne avec le cout du plein en tenant compte du trajet et du prix


def prix_plein(Quantité, adresse, station) : 
    position=Adresse_to_gps(adresse)
    A = list(map(float,position.split(',')))
    B = list(map(float,df_opti.iloc[station,2].split(',')))
    
    D= distance1(A,B)     #calcul de la distance avec la station (marche pas donc j'ai mis une constante)
    conso_trajet=D*0.06 #calcul du nb de litre d'essence necessaire pour aller a la station
    P=Quantité+conso_trajet #calcul du nb de litre necessaire pour faire le plein après le trajet
    prix_L=df_opti.iloc[station,3] #prix au litre à la station
    prix_trajet=D*conso_trajet #prix du trajet
    px_plein=prix_L*P-prix_trajet #prix du plein
    return px_plein



In [26]:
#df_opti=df_test.assign(cout=1) #nouvelle colonne avec le cout du plein en tenant compte du trajet et du prix

for i in range(len(df_opti)) : 
    C=prix_plein(50,"1 rue d'Arsonval, Orsay, France",i)
    df_opti.iloc[i,5]=C

df_opti = df_opti.sort_values('cout', ascending=True)
print(df_opti.head(1)) #station la plus rentable

                       adresse       ville              geom  prix_valeur  \
44865  38 rue du moulin à vent  VéNISSIEUX  45.73012,4.85621        1.709   

      prix_nom          cout  
44865     SP95 -2.126759e+13  
